In [1]:
import os
import traceback
import pandas as pd
import geopandas as gpd
import sqlalchemy
import matplotlib.pyplot as plt
import numpy as np

user='hifld_summer'
pw='hifld_summer'
name='hifld_summer'
host='moria'
port=2023
engine = sqlalchemy.create_engine(
    f'postgresql://{user}:{pw}@{host}:{port}/{name}'
    )

In [30]:
%%time
sql ="""
select build_id, geom, prim_occ from structures.alabama limit 500000
"""
try:
    gdf = gpd.read_postgis(sql, con=engine, geom_col='geom') # epsg 4326

    gdf = gdf.to_crs("ESRI:102003") # USA_Contiguous_Albers_Equal_Area_Conic Projection

    vertex_count = []

    for i, row in gdf.iterrows():
        if row.geom.geom_type.startswith("Multi"):
            n = 0
            for part in row.geom.geoms:
                n += len(part.exterior.coords) - 1
        else:
            n = len(row.geom.exterior.coords) - 1
        vertex_count.append(n)

    gdf["vertex_count"] = vertex_count

except Exception:
    print(traceback.print_exc())

CPU times: total: 13.7 s
Wall time: 13.9 s
